# CASE STUDY: VISA RUPAY CREDIT CARD COMPANY

## VisaRuPay Credit Card Company

VisaRuPay is a consumer credit card company that has a large number of customers (or accounts). These customers charge some of their purchases on their VisaRuPay cards. The charges made in one month are due by the end of the next month. If a customer fails to make
the minimum payment in a given month, the company flags the account as delinquent. The company keeps track of the payment history of each customer so that it can identify customers who are likely to default on their obligations and not pay their debt to the company.

### STATE DESCRIPTIONS

Customer's State (or stage) k: if he or she has missed making the minimum payment for the last k consecutive months.

#### Four possible futures: 
a) Minimum payment (or more) and move to stage 0. </br> 
<br> b) Make no payment (or less than the minimum payment) and move to stage k+1. </br>
<br> c) Default by declaring bankruptcy, thus moving to stage D or the company can cancel the customer’s card and terminate the account. (state C). </br> 

#### Current Policy (Pc)

Currently the company has a simple policy: it terminates an account as soon as it misses seven minimum payments in a row and writes off the remaining outstanding balance on that account as a loss.


  | k | 0 | 1 | 2 | 3 | 4 | 5 | 6 |
  | --- | --- | --- | --- | --- | --- | --- | --- |
  | pk | 0.033 | 0.048 | 0.090 | 0.165 | 0.212 | 0.287 | 0.329 |
  | qk | 0.030 | 0.021 | 0.037 | 0.052 | 0.075 | 0.135 | 0.182 |
  | bk | 1243.78 | 2090.33 | 2615.16 | 3073.13 | 3502.99 | 3905.77 | 4280.26 |

where,

pk:  Probability that a customer in state k fails to make the minimum payment in the current period and thus moves to state k+1. </br>
<br> qk:  Probability that a customer in state k declares bankruptcy in the current period and thus moves to state D. </br>
<br> bk:  Average outstanding balance of a customer in state k. </br>


In [50]:
# IMPORTING THE NECESSARY LIBRARIES
import numpy as np
from pydtmc import MarkovChain as mc

In [51]:
dir(mc)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_absorbing_states_indices',
 '_calculate_period',
 '_communicating_classes_indices',
 '_create_rng',
 '_cyclic_classes_indices',
 '_cyclic_states_indices',
 '_gth_solve',
 '_recurrent_classes_indices',
 '_recurrent_states_indices',
 '_slem',
 '_states_indices',
 '_transient_classes_indices',
 '_transient_states_indices',
 'absorbing_states',
 'absorption_probabilities',
 'absorption_times',
 'accessibility_matrix',
 'adjacency_matrix',
 'are_communicating',
 'backward_committor',
 'backward_committor_probabilities',
 'birth_death',
 'communicating_classes',
 'conditional_distribution',
 'conditional_probabilities',
 'cyc

###  Question
1) Build a stochastic model using DTMC to help the management of VisaRuPay analyze the performance of this policy in a rational way. Assume that the state of an account changes in a Markov fashion and when a customer account is terminated or the customer declares bankruptcy, it simply replaces that account with an active one, so that number of accounts does not change. 

## TRANSITION PROBABILITY MATRIX

In [64]:
p = [0.033,0.048,0.090,0.165,0.212,0.287,0.382]
p_bar = [round((1-i),3) for i in p]
print(p_bar)
declare_bankruptcy = [0.30,0.021,0.037,0.052,0.075,0.135,0.182]
out_bal = [1243.78,2090.33,2615.16,3073.13,3502.99,3905.77,4280.26]

n_states = 7
# Defining the TPM
TPM = []
for i in range(n_states):
    l = []
    if i == 6:
        l.append(p_bar[i]+p[i])
    else:
        l.append(p_bar[i])
    for j in range(1,n_states):
        if j==i+1:
            l.append(p[i])
        else:
            l.append(0)
    
    TPM.append(l)
print(TPM)

[0.967, 0.952, 0.91, 0.835, 0.788, 0.713, 0.618]
[[0.967, 0.033, 0, 0, 0, 0, 0], [0.952, 0, 0.048, 0, 0, 0, 0], [0.91, 0, 0, 0.09, 0, 0, 0], [0.835, 0, 0, 0, 0.165, 0, 0], [0.788, 0, 0, 0, 0, 0.212, 0], [0.713, 0, 0, 0, 0, 0, 0.287], [1.0, 0, 0, 0, 0, 0, 0]]


In [66]:
# Stochastic process

process = mc(TPM, ['0','1','2','3','4','5','6'])
print(process)


DISCRETE-TIME MARKOV CHAIN
 SIZE:         7
 CLASSES:      1
  - RECURRENT: 1
  - TRANSIENT: 0
 ABSORBING:    NO
 APERIODIC:    YES
 IRREDUCIBLE:  YES
 ERGODIC:      YES



## Question

2. Analyze the current policy (Pc), which terminates an account as soon as it misses the seventh minimum payment in a row by assuming expected annual loss due to bankruptcies and account closures as performance measure.

In [81]:
# STATIONARY DISTRIBUTION 

stat_dist = (process.pi)
print(stat_dist[0])

[9.66410938e-01 3.18915610e-02 1.53079493e-03 1.37771543e-04
 2.27323046e-05 4.81924858e-06 1.38312434e-06]
